In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [ ]:
#page & w 교체 필요
# 월드 목록 : 오로라(3), 레드(4), 이노시스(5), 유니온(6), 스카니아(7), 루나(8), 제니스(9), 크로아(10), 베라(11), 엘리시움(12), 아케인(13), 노바(14)
# 월드 목록을 리스트로 만들어서 world 번호를 반복으로 돌릴 수 있지만, 데이터 수집에 너무 많은 시간이 걸리는 이유로 그러지 않았습니다.

headers = {
    'User-Agent': 'chrome 이용해서 user agent 정보 입력'
}

# 조회하고자 하는 페이지
page = 1
# 조회하고자 하는 월드 번호
w = 3
check = True
data = []

while True:
    # 랭킹 페이지
    url = f"https://maplestory.nexon.com/N23Ranking/World/Total?page={page}&w={w}"
    res = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(res.text, "html.parser")
    
    time.sleep(2)
    
    lv_data = []
    name_data = []
    
    lv_temp = soup.find("tbody").find_all("tr")
    for lv in lv_temp:
        # 260레벨 이상 데이터로 필터링
        if int(lv.find_all("td")[2].text[3:]) < 260:
            check = False
            break
        else:
            lv_data.append(lv.find_all("td")[2].text[3:])
    
    if soup.find("tbody").find_all("dt"):
        items = soup.find("tbody").find_all("dt")
        
        for item in items:
            name_data.append(item.find("a").text)
    
    else:
        print("마지막 페이지입니다.")
        break
    
    for i in range(len(lv_data)):
        data.append([lv_data[i], name_data[i]])
    
    # 한 페이지당 10초 정도 쉬어주지 않으면 ip밴을 당할 수 있음
    time.sleep(8)
    
    # 경과 확인용 출력 (필수 아님!)
    print(page, len(data))
    
    # 백업용 데이터 생성 (필수 아님!)
    if page % 500 == 0:
        temp = pd.DataFrame(data)
        temp.to_csv(f"temp{page}.csv")
        
    if check:
        page += 1
    else:
        break

In [63]:
df = pd.DataFrame(data)
df.rename(columns={0:"LV", 1:"닉네임"},inplace=True)
df.to_csv("data.csv")